In [6]:
# import libraries
# DataFrame
import pandas as pd

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# sklearny
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB


#tensorflow
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

# Utility
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import string
import pickle
# RNN - LSTM
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import re
import keras
from keras import regularizers
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Test Input dataset before model training

In [7]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))
print(stopword)

{'they', "he'd", 'against', "you'll", 'all', 'same', "mustn't", 'ma', 'myself', 'are', 'of', 'am', "it'd", 'there', 'not', 'y', 'had', 'their', 'own', 'having', 'haven', 'very', "doesn't", 'm', 'only', 'shouldn', 'below', 'off', 'did', 's', "he'll", 'have', 'other', 'been', 'until', 'if', 'yourself', 'up', 'an', 'we', 'herself', "that'll", 'his', 'has', "you're", 'and', 'mustn', 'my', 'out', 'it', 'then', "we've", "hadn't", 'i', 'do', "i'd", 'you', 'her', "aren't", 'those', 'won', 'each', 'that', 'its', 'your', 'him', 'because', 'but', "isn't", 'd', 'after', 'why', 'will', "haven't", 'nor', "they'll", 'on', 'them', 'be', 'should', "weren't", 'between', 'a', 'themselves', 'were', 'll', 'weren', 'he', 'who', 'about', 'our', "didn't", 'how', 'above', 'wasn', 'yours', "couldn't", "hasn't", 'most', 'through', 'as', 'didn', 're', "should've", 'the', 'isn', 'yourselves', 'is', 'too', 'ain', 'aren', "it'll", 'some', 'few', 'o', "mightn't", 'hers', 'no', 'being', 'was', 'over', 'mightn', 'doesn

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/claudiachen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern = '@[^\s]+'
def process_tweets(tweet):
  # Lower Casing
    tweet = tweet.lower()
    #tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    #Removing Stop Words
    negation_words = {"no", "nor", "not", "don", "don't", "didn", "didn't", "doesn", "doesn't", "can't", "cannot", "won't", "wouldn", "wouldn't", "shouldn", "shouldn't", "isn't", "aren't", "wasn't", "weren't"}

    # Remove negation words from the stop word list
    custom_stop_words = stopword.difference(negation_words)
    final_tokens = [w for w in tokens if w not in custom_stop_words]
    #final_tokens = tokens
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

def process_tweets_append(tweet):
  # Lower Casing
    tweet = tweet.lower()
    #tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    #Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    final_tokens = tokens
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    negation_words = {"not", "no", "never", "n't"}
    prev_neg = False

    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        #finalwords.append(word)
        if word in negation_words:
          prev_neg = True
        elif prev_neg:
          finalwords.append("not_" + word)
          prev_neg = False
        else:
          finalwords.append(word)
    return ' '.join(finalwords)

In [9]:
def tokenizer(max_words, test_text):
    # 1. build word table with top 50 words
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(test_text)
    print("单词表:", tokenizer.word_index)

    sequences = tokenizer.texts_to_sequences(test_text)
    print("文本的单词索引序列:", sequences)

    tweets = pad_sequences(sequences, maxlen=10)
    print(tweets.shape)
    print("填充后文本的单词索引序列:", tweets)

In [10]:
# Test
max_words = 20
test_tweets = [
    "I LOVE the movie!!!@user123",
    "Not happy with the service at all",
    "Never coming back! website at https://www.example.com",
    "Not happy with the service at all."
]

processed_test_text = [process_tweets(x) for x in test_tweets]
print("预处理后的数据:", processed_test_text)
tokenizer(max_words, processed_test_text)

预处理后的数据: ['love movie', 'not happy service', 'never coming back website', 'not happy service']
单词表: {'not': 1, 'happy': 2, 'service': 3, 'love': 4, 'movie': 5, 'never': 6, 'coming': 7, 'back': 8, 'website': 9}
文本的单词索引序列: [[4, 5], [1, 2, 3], [6, 7, 8, 9], [1, 2, 3]]
(4, 10)
填充后文本的单词索引序列: [[0 0 0 0 0 0 0 0 4 5]
 [0 0 0 0 0 0 0 1 2 3]
 [0 0 0 0 0 0 6 7 8 9]
 [0 0 0 0 0 0 0 1 2 3]]
